In [28]:
import pymongo
import time
import re
import hashlib
import random
import requests
import numpy as np
from tqdm import *
from fake_useragent import UserAgent
from urllib.error import URLError
from urllib.request import ProxyHandler, build_opener
from bson.objectid import ObjectId
import DXYspider_Func as sp

In [29]:
def get(page):
    if page>3:
        response= requests.get('http://www.kxdaili.com/dailiip/2/1.html',timeout=5)
    else:
        response= requests.get(f'https://www.beesproxy.com/free/page/{page}',timeout=5)
    response.close()
    html = response.text
    return(html)
def parse(html):
    
    pattern = re.compile(
    f'<tr.*?>.*?<td>(.*?)</td>.*?<td>(.*?)</td>.*?</tr>',re.S)
    items = re.findall(pattern, html)
    return items

In [30]:
response= requests.get('http://www.ip3366.net/?stype=1&page=1',timeout=5)
html = response.text
a = parse(html)
print(a)

[('190.89.38.49', '999'), ('45.86.220.239', '443'), ('182.87.153.121', '8118'), ('165.154.2.225', '3128'), ('175.6.185.156', '9128'), ('115.211.39.171', '9000'), ('182.160.12.153', '443'), ('111.225.153.248', '8089'), ('218.1.200.97', '57114'), ('222.162.77.230', '7890')]


In [31]:
client = pymongo.MongoClient('localhost', 27017)
db = client.PROXY
collection1 = db.proxies

In [16]:
def update_proxy(collection,Timespan):                                 #填入数据表、IP生命周期，更新代理，返回 可用IP数
    ipf = 0
    ipu = 0
    print('updating Proxy')
    for page in range(0,4):                         #获取、验证代理ip
        html = get(page+1)
        ips= parse(html)
        rang = len(ips)
        ipf += rang
        count = 0
        for i in tqdm(range(rang)):
            proxies = {
                'https':'https://'+ips[i][0]+':'+ips[i][1],
                'http':'http://'+ips[i][0]+':'+ips[i][1]
                }
            headers = {
            'Connection': 'close'
            }

            try:
                response = requests.get('http://httpbin.org/get',headers = headers ,proxies = proxies,timeout = 3)
                response.close()
                if response.status_code ==200 :
                    result = {'ip':f'{ips[i][0]}','port':f'{ips[i][1]}','usage' : Timespan}
                    collection.insert_one(result)
                    count += 1
                else:
                    continue
            except requests.exceptions.RequestException as r:
                continue
            except requests.exceptions.ConnectionError as e:
                continue
        ipu += count
    print(f'{ipf} ips found,{ipu} ips available.')
    return count

In [17]:
def check(collection):                                                    #检查，删除Timespan=0的IP，返回可用IP、删除的ip数量                                              
    condition1 = {'usage': {'$lte':0} }   
    condition2 = {'usage': {'$gt':0} }
    deleted = collection.delete_many(( condition1 ))    #筛选timespan>0 IP
    proxyNum = collection.count_documents( condition2 )

    return proxyNum,deleted.deleted_count

In [18]:

def get_ip(collection):                                       #从Timespan>0的IP中随机选取一个并减少其Timespan
    condition = {'usage': {'$gt':0} }           
    proxyList = list(collection.find( condition )) #筛选timespan>0 IP

    if len(proxyList) <= 0:
        return 1
    rand = random.randint(0,len(proxyList)-1)
    proxyId = {'_id':(proxyList[rand].get('_id'))}            #记录选择的ID

    proxies = {'http':'http://' + proxyList[rand].get('ip')+':'+proxyList[rand ].get('port') }
    collection.update_one(proxyId ,{"$inc":{"usage":-1}} )    #更新IP的usage
    return proxies

In [19]:
def proxy(collection):
    Timespan = 200
    ipStatus = check(collection)
    if ipStatus[0] >0:
        proxies = get_ip(collection)
        if proxy != 1:
            return proxies
    update_proxy(collection,Timespan)
    proxies = proxy(collection)
    return proxies

In [34]:
update_proxy(collection1,40)

updating Proxy


100%|██████████| 10/10 [00:16<00:00,  1.62s/it]

70 ips found,23 ips available.


5